In [ ]:
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split

data, target = fetch_california_housing(return_X_y=True, as_frame=True)
target *= 100  

data_train, data_test, target_train, target_test = train_test_split(
    data, target, random_state=42
)

In [2]:
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsRegressor
from sklearn.pipeline import Pipeline

model = Pipeline([
    ("scaler", StandardScaler()),
    ("knn", KNeighborsRegressor())
])

In [3]:
from sklearn.model_selection import RandomizedSearchCV
import numpy as np

param_grid = {
    "knn__n_neighbors": np.logspace(0, 3, num=20).astype(int),   # number of neighbors
    "scaler__with_mean": [True, False],                          # scaling options
    "scaler__with_std": [True, False],                           # scale std or not
}

search = RandomizedSearchCV(
    model,
    param_grid,
    n_iter=20,
    scoring="neg_mean_absolute_error",
    random_state=42
)

search.fit(data_train, target_train)
best_params = search.best_params_
best_params

{'scaler__with_std': True,
 'scaler__with_mean': False,
 'knn__n_neighbors': np.int64(6)}

In [7]:
import pandas as pd
# use the already-fitted RandomizedSearchCV object `search`
cv_results = pd.DataFrame(search.cv_results_)
cv_results["mean_test_score"] *= -1
column_name_mapping = {
    "param_knn__n_neighbors": "n_neighbors",
    "param_scaler__with_mean": "centering",
    "param_scaler__with_std": "scaling",
    "mean_test_score": "mean test score",
}

cv_results = cv_results.rename(columns=column_name_mapping)
cols = ["n_neighbors", "centering", "scaling", "mean test score"]
cv_results = cv_results[cols].sort_values("mean test score")
cv_results

,n_neighbors,centering,scaling,mean test score
2,6,False,True,44.230674
0,12,False,True,44.620616
5,12,True,True,44.620616
4,4,False,True,45.003390
19,37,True,True,46.324784
6,2,False,True,47.447982
9,2,True,True,47.447982
8,1,True,True,53.636768
1,1,True,True,53.636768
14,483,True,True,56.861021
